# Example notebook for Worst-of-DOP
We model the worst-of down and out Put using the Rainbow-Specification.

In [ ]:
import datetime
import pylab as pl
import matplotlib.pyplot as plt
import pyvacon.analytics as analytics
import pyvacon.tools.enums as enums
import pyvacon.marketdata.testdata as mkt_testdata
import pyvacon.environment as environment
import pyvacon.models.tools as model_tools
import pyvacon.tools.converter as converter
import pyvacon.pricing.tools as pricing_tools
import pyvacon.instruments.tools as ins_tools
import numpy as np
import pylab as pl
%matplotlib inline

## Get testdata

In [ ]:
refdate = analytics.ptime(2017,7,1,0,0,0)

env_test = environment.CalculationDataEnvironment()
mkt_testdata.InterestRate.all_data(env_test)
mkt_testdata.Equity.all_data(env_test)
spot1 = env_test.mktman.getForwardCurve('DBK').value(refdate,refdate)
spot2 = env_test.mktman.getForwardCurve('EON').value(refdate,refdate)

## Setting up the specification
In this section we show how to setup an Equita Monte Carlo product using the rainbow specification as provided by the nalytics library. 
### Payoff description
A Worst-of Down&Out put is a put on the worst of a basket of stocks $S_i$, $1\leq i\leq N$, i.e. it has payoff
$$
P(S_1,...,S_N) = \left\{ \begin{array}{l} (K-min_i(w_iS_i(T)))_+ \mbox{ if }min_{0\leq t\leq T} min_i(w_iS_i(t))> b, \\ r \mbox{ otherwise. }\end{array}\right. 
$$
for a given barrierlevel $b$, rebate $r$ and strike $K$ and basket weights $w_i$.

In the following we consider the case $K=100$, $r=0$, $b=80$ and the weights are chosen as $w_i=1.0/S_i(0)$.

### The rainbow specification
A rainbow specification consists of a set of barriers and rainbow basket underlyings.
#### Rainbow basket underlying
A rainbow basket underlying $u$ is constructed by first weighting the single constituents according to their basket weights, applying individual caps and floors. Then, a weighted sum of the capped and floored values is computed, these capped and floored single values are sorted from worst to best and are then summed up. In addition, one can define a forward start time together with forward start type, i.e. if the weights are adjusted by individual factors w.r.t. the single spots or by the basket value.
To realize asian features one can also specify ficing dates together with different time aggregation types.
#### Baskets
Each barrier is defined by
- A rainbow underlying id to the underlying on which the barrier is monitored.
- A lower and upper level describing the range when the barrier is hit.
- A start and end date describing the begin and end of the monitoring. If start equals end, a discrete barrier is constructed, if a list of monitoring dates is given, the barrier is only monitored at these discrete points.
- A piecewise linear payoff if barrier is hit.
- A piecewise linear payoff if barrier is not hit.
- A flag whethr the barrier is active.
- A list of barriers which are activated/inactivated by hitting this barrier.
- A paydate deining when payoffs are paid. To define payoff at hit, the paydate has to be set before the barrir start date.

### Example: Wof DOP
#### The down & out barrier
We first define the down and out barrier. This barrier pays zero at hit and must switch off the barrier which defines the final Put payoff.

In [ ]:
barrier_specs = analytics.vectorRainbowBarrierSpec(2)
expiry = analytics.ptime(2018,7,1,0,0,0)
barrier_start = refdate
barrier_end = expiry
obs_dates = analytics.vectorPTime()
lower_level = -1000
upper_level = 80
udl_id = 0
# switch of the final "payoff" barrier if barrier is hit
barrier_num = analytics.vectorInt(1)
barrier_num[0] = 1
barrier_status = analytics.vectorBool(1)
barrier_status[0] = False
is_active = True
hit_payoff_points = analytics.vectorDouble([0.0, 800000.0])
hit_payoff_values = analytics.vectorDouble([0.0, 0.0])
no_hit_payoff_points = analytics.vectorDouble()
no_hit_payoff_values = analytics.vectorDouble()
paydate = analytics.ptime(2000,1,1,0,0,0) #if paydate < startdate, then payoff is at hit
barrier_specs[0] = analytics.RainbowBarrierSpec(barrier_start, barrier_end, obs_dates, lower_level, upper_level, udl_id, 
                                            barrier_num, barrier_status, is_active, hit_payoff_points, hit_payoff_values, 
                                              no_hit_payoff_points,  no_hit_payoff_values, paydate)

### The barrier to model the final Put payoff

In [ ]:
refdate = analytics.ptime(2017,7,1,0,0,0)
expiry = analytics.ptime(2018,7,1,0,0,0)
barrier_start = expiry
barrier_end = expiry
obs_dates = analytics.vectorPTime()
lower_level = -100000 #determine barrier levels so that barrier is hit with probability 1
upper_level = 1000000
udl_id = 0
strike = 100
barrier_num = analytics.vectorInt()
barrier_status = analytics.vectorBool()
hit_payoff_points = analytics.vectorDouble([0, strike, strike+1000000])
hit_payoff_values = analytics.vectorDouble([0,0, 1000000])
no_hit_payoff_points = analytics.vectorDouble()
no_hit_payoff_values = analytics.vectorDouble()
paydate = expiry
barrier_specs[1] = analytics.RainbowBarrierSpec(barrier_start, barrier_end, obs_dates, lower_level, upper_level, udl_id, 
                                            barrier_num, barrier_status, is_active, hit_payoff_points, hit_payoff_values, 
                                              no_hit_payoff_points,  no_hit_payoff_values, paydate)

### The worst-of basket specification

In [ ]:
udls = analytics.vectorRainbowUdlSpec(1)
basket = analytics.vectorString(['DBK','EON'])
underlying_weights = analytics.vectorDouble([100.0/spot1,100.0/spot2])
underlying_caps = analytics.vectorDouble()#[1000000,1000000])
underlying_floors = analytics.vectorDouble()#[-10000,-100000])
weights_after_sort = analytics.vectorDouble([1.0,0])
basket_cap = analytics.getMaxDouble('');
basket_floor = -analytics.getMaxDouble('');
additive_offset = 0.0
fixing_dates = analytics.vectorPTime()
fwd_value_type = 'NONE' # set the fwd-start type (here: no fwd start), possible values: None, FwdStartSingle, FwdStartBasket, FloatingStrikeBasket
fwd_time_agg_type = 'NONE' # set fwd start reference type, possible values: None, Min, Max, Mean
type_str = 'WorstOf' # just for performance and validation, the type of basket: None, Basket, WorstOf, BestOf, General, BasketOfPerformances,
                    #PerformanceOfBasket, Asian, LookbackBestOf, LookbackWorstOf
fwd_start_fixings = analytics.vectorPTime()
time_agg_type = 'None'
floating_strike = 0
udls[0] = analytics.RainbowUnderlyingSpec(basket, underlying_weights, underlying_caps, underlying_floors, weights_after_sort, 
                                          basket_cap, basket_floor, additive_offset, fwd_value_type, fwd_time_agg_type, 
                                          fwd_start_fixings,floating_strike,time_agg_type, fixing_dates, type_str)



### The Specification

In [ ]:
spec = analytics.RainbowSpecification('WOF_DOP', 'DEKA', enums.SecuritizationLevel.NONE, 'EUR', expiry, barrier_specs, udls)

## Pricing with local volatility
### Setting up PricingData

In [ ]:
pricing_data = analytics.LocalVolMonteCarloPricingData()
pricing_data.valDate = refdate
pricing_data.spec = spec
pricing_data.dsc = mkt_testdata.InterestRate.get_curve('EONIA', refdate)
pricing_data.param = analytics.MonteCarloPricingParameter()
pricing_data.param.mcParam.numberOfSimulations = 10000
vols = analytics.vectorConstVolatilities(2)
vols[0] = env_test.mktman.getVolatilitySurface('DBK')
vols[1] = env_test.mktman.getVolatilitySurface('EON')
pricing_data.vols = vols
correlation = analytics.vectorVectorDouble(2)
correlation[0] = analytics.vectorDouble([1,0.6])
correlation[1] = analytics.vectorDouble([0.6,1])
pricing_data.setCorrelations(correlation)
pricing_data.pricingRequest = analytics.PricingRequest()

### Pricing

In [ ]:
analytics.registerSerialization('dummy')
tic = datetime.datetime.now()
#for i in range(1000000):
#pricing_data.save('WOF.json',pricing_data)
pr = analytics.price(pricing_data)
print('runtime: {}'.format(datetime.datetime.now() - tic))
#plot the price
pr.getPrice()

### Pricing with Local Correlation

In [ ]:
correlation_high = analytics.vectorVectorDouble(2)
correlation_high[0] = analytics.vectorDouble([1,0.99])
correlation_high[1] = analytics.vectorDouble([0.99,1])
moneyness = analytics.vectorDouble(pl.frange(0.5,1.5,0.005))
moneyness_scale = analytics.vectorDouble([100, 100])
local_correlation_model = analytics.CorrelationModelLocalPairwise(correlation, correlation_high, moneyness_scale, moneyness)

def local_correlation_function(moneyness, t):
    result = analytics.vectorVectorDouble(len(moneyness))
    kappa = 100
    for i in range(len(moneyness)):
        tmp = analytics.vectorDouble(len(moneyness))
        for j in range(len(moneyness)):
            tmp[j] = 1.0 - np.exp(-kappa*(moneyness[i]-moneyness[j])*(moneyness[i]-moneyness[j]))
        result[i] = tmp
    return result

local_corr_function = local_correlation_function(moneyness, 1.0)
#local_corr_function = local_correlation_function(moneyness, 3.0)
local_correlation_model.addLocalCorrelationFunction(0.1, local_corr_function)
pricing_data.correlationModel = local_correlation_model
tic = datetime.datetime.now()
analytics.registerSerialization('depp')
#pricing_data.save('WOF_local_corr.json',pricing_data)
pr = analytics.price(pricing_data)
print('runtime: {}'.format(datetime.datetime.now() - tic))
print(pr.getPrice())

### Cashflow-Exposure
For XVA calculations the future exposures are needed. These exposures at a given time are typically computed from all simulated future cashflows. The MC pricer provides functionality to compute these future cashflows. One has simply to set the times at which the future cashflows are needed into the PricingRequest object. The PricingResults will then contain the PV of the cashflows woch occured between the defined cashflow periods.

In [ ]:
rebate = 10
pricing_data.spec = ins_tools.create_wof_dop(refdate, [(100.0/spot1, 'DBK'), (100.0/spot2,'EON')], 365, 80, 120, rebate)
pricing_data.pricingRequest.setCashflowTimes(converter.createPTimeList(refdate, range(30,366, 30)))
#pricing_data.save('WOF_local_corr.json',pricing_data)
pr = analytics.price(pricing_data)

In [ ]:
print(pr.getCashflowSlices().cashflowSlices.size())
future_cash = pricing_tools.cashflow_profile(pr)

In [ ]:
mean_exposure = []
quantile_exposure = []
for x in future_cash:
    mean_exposure.append(np.mean(x))
    quantile_exposure.append(np.percentile(x,60))
#n, bins, patches = plt.hist(future_cash[3], 50, normed=1, facecolor='blue', alpha=0.75)
plt.plot(mean_exposure,'-x')
plt.plot(quantile_exposure,'-^')

## Pricing with stochastic volatility
### Setting up PricingData
#### Market data

In [ ]:
stochvol_pricing_data = analytics.StochasticVolMonteCarloPricingData()
stochvol_pricing_data.valDate = refdate
stochvol_pricing_data.spec = spec
stochvol_pricing_data.dsc = mkt_testdata.InterestRate.get_curve('EONIA', refdate)
stochvol_pricing_data.param = analytics.MonteCarloPricingParameter()
stochvol_pricing_data.param.mcParam.numberOfSimulations = 10000
stochvol_pricing_data.vols = analytics.vectorConstVolatilities(2)
stochvol_pricing_data.vols[0] = env_test.mktman.getVolatilitySurface('DBK') #volatilities are just needed to get current forward curve for Buehle model
stochvol_pricing_data.vols[1] = env_test.mktman.getVolatilitySurface('EON')

asset_correlations = np.array([ [ 1.0, 0.8 ],
                                 [ 0.8, 1.0] ])
stochvol_pricing_data.setCorrelations(converter.from_np_matrix(asset_correlations))
stochvol_pricing_data.pricingRequest = analytics.PricingRequest()

#### Setup Heston model

In [ ]:
S0 = 1.0 # we model te Buehler X-Process, therefore we start at 1.0
kappa = 1.0
theta = 0.04
alpha = 0.010
v0 = 0.03
rho = -0.8
pricing_models = [ analytics.HestonModel('HESTON_DAX', refdate, S0, v0, theta, kappa, alpha, rho), analytics.HestonModel('HESTON_STOXX50E', refdate, S0, v0, theta, kappa, alpha, rho)]
stochvol_pricing_data.models = analytics.vectorConstModel(pricing_models)

### Pricing

In [ ]:
tic = datetime.datetime.now()
analytics.setLogLevel('DEBUG')
#stochvol_pricing_data.save('WOF_stoch_vol.json',stochvol_pricing_data)
pr = analytics.price(stochvol_pricing_data)
print('runtime: {}'.format(datetime.datetime.now() - tic))
#plot the price
pr.getPrice()

### Projection of model parameters

In [ ]:
import pyvacon.pricing.tools as pricing_tools
model_param = 'speed of meanreversion'
heston_proj = analytics.HestonModel('HESTON_DAX', refdate, S0, v0, theta, kappa, alpha, rho)
stochvol_pricing_data.models[0] = heston_proj
prices, x, index = pricing_tools.project_model_param(stochvol_pricing_data, heston_proj, model_param, 0.0,2.5,0.1)

In [ ]:
plt.plot(x,prices,'-x')
plt.xlabel(model_param)
plt.ylabel('price')

### Correlation Analysis

In [ ]:
corr, s_v_corr, v_v_corr = model_tools.compute_stoch_vol_correlations(pricing_models, [4,4], asset_correlations, 0.0, 0.0)
stochvol_pricing_data.setSpotVarianceCorrelations(converter.from_np_matrix(s_v_corr))
stochvol_pricing_data.setVarianceVarianceCorrelations(converter.from_np_matrix(v_v_corr))

In [ ]:
tic = datetime.datetime.now()
analytics.setLogLevel('DEBUG')
pr = analytics.price(stochvol_pricing_data)
print('runtime: {}'.format(datetime.datetime.now() - tic))
#plot the price
pr.getPrice()